## Library imports, data imports, and initialisations

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
%matplotlib inline

import seaborn as sns
color = sns.color_palette()

import warnings
warnings.filterwarnings('ignore')

from scipy.special import boxcox1p
from scipy.stats import norm, probplot
from sklearn.preprocessing import StandardScaler

## Load in datasets
Read the CSV files for the training and testing data, then create a concatenated version for feature engineering.

In [ ]:
# Load in the training and testing datasets
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# Save and remove the ID columns
trainingIDs = df_train['Id']
testingIDs = df_test['Id']

df_train.drop("Id", axis = 1, inplace = True)
df_test.drop("Id", axis = 1, inplace = True)

# Combine the datasets
df_allData = pd.concat((df_train, df_test)).reset_index(drop = True)
df_allData.drop(['SalePrice'], axis = 1, inplace = True)

## Impute missing values
I've chosen to do this first, so that any effects it has on relationships or other aspects of this exploration/modeling can be accounted for early on.

In [ ]:
def getMissingValues():
    missingValuesTotal = df_allData.isnull().sum().sort_values(ascending = False)
    missingValuesPercent = (100 * df_allData.isnull().sum() / df_allData.isnull().count()).sort_values(ascending = False)

    missingValues = pd.concat([missingValuesTotal, missingValuesPercent], axis = 1, keys = ['# Missing', '% Missing'])
    return missingValues[missingValues['# Missing'] > 0]

getMissingValues()

In [ ]:
# Fill values in the training and testing datasets for the column specified
def fillMissingValues(var, fill):
    #df_train[var] = df_train[var].fillna(fill)
    #df_test[var] = df_test[var].fillna(fill)
    df_allData[var] = df_allData[var].fillna(fill)
    
# PoolQC w/ missing values: 99.66%
# In data_description.txt, NaN values indicate there is no pool
fillMissingValues('PoolQC', 'NA')

# MiscFeature w/ missing values: 96.29%
# In data_description.txt, NaN values indicate there are no misc features
fillMissingValues('MiscFeature', 'NA')

# Alley w/ missing values: 93.75%
# In data_description.txt, NaN values indicate there is no alley access
fillMissingValues('Alley', 'NA')

# Fence w/ missing values: 80.76%
# In data_description.txt, NaN values indicate there is no fence
fillMissingValues('Fence', 'NA')

# FireplaceQu w/ missing values: 47.42%
# In data_description.txt, NaN values indicate there is no fireplace
fillMissingValues('FireplaceQu', 'NA')

# LotFrontage w/ missing values: 17.8%
# In data_description.txt, no substitution is given for NaN LotFrontage characteristics; thus I'm going to use the median values for the row's corresponding neighbourhood.
# This may have issues when a there is a NaN value in a neighbourhood with no other samples, but for now this is not the case.
df_allData['LotFrontage'] = df_allData.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# GarageCond, GarageFinish, GarageType, GarageQual, GarageYrBlt w/ missing values: 5.57%;
# In data_description.txt, NaN values indicate there is no garage
for cVar in ['GarageCond', 'GarageType', 'GarageFinish', 'GarageQual']:
    fillMissingValues(cVar, 'NA')
fillMissingValues('GarageYrBlt', 0)

# BsmtExposure, BsmtFinType2 w/ missing values: 2.61%; BsmtCond, BsmtFinType1, BsmtQual w/ missing values: 2.54%
# In data_description.txt, NaN values indicate there is no basement
for cVar in ['BsmtExposure', 'BsmtFinType2', 'BsmtCond', 'BsmtFinType1', 'BsmtQual']:
    fillMissingValues(cVar, 'NA')

# MasVnrArea, MasVnrType w/ missing values: 0.55%
# In data_description.txt, NaN values indicate there is no masonry vaneer
fillMissingValues('MasVnrArea', 0)
fillMissingValues('MasVnrType', 'NA')

# Electrical w/ missing values: 0.07%
# In data_description.txt, no substitution is given for NaN Electrical characteristics; thus I assume the one missing value is an error to be dropped.
df_allData = df_allData.drop(df_allData[df_allData['Electrical'].isnull()].index)

# Confirm that all missing values have been fixed
getMissingValues()

## Clean data types
Change all categorical columns are as noted as numbers to strings.

In [ ]:
cols_numToObj = ['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt',
                 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

for col in cols_numToObj:
    df_train[col] = df_train[col].apply(str)
    df_test[col] = df_test[col].apply(str)

## Preliminary correlation inspections
Review correlation values between each set of characteristics, to identify if there are any redundant columns that could be dropped.

In [ ]:
# Correlation matrix for all characteristics
corrMat = df_train.corr()

# Determine which correlations are significant, and drop the others
# Assuming correlation values >= 0.8 are significant
corrValues = corrMat.unstack().abs()
c = [corrValues.drop(i, inplace = True) for i, v in corrValues.items()
     if i[0] == i[1]                           # left and right index are the same
     or v < 0.8                                # Value is insignificant
     or (i[1], i[0]) in corrValues.index]      # repeated correlation values

# Sort and print the correlation values
print('Characteristics with significant correlations:\n{}\n'.format(corrValues.sort_values(ascending = False)))

# Print the figure
sns.set(font_scale = 1.0, rc = {'figure.figsize': (12, 10)})
sns.heatmap(corrMat, vmax = 0.8, square = True)

Very strong correlations between the following:
* GarageArea & GarageCars
* TotRmsAbvGrd & GrLivArea
* 1stFlrSF & TotalBsmtSF

I'll check which characteristic in each of the above pairs has a stronger correlation with SalePrice, and remove the other column.

In [ ]:
# Saleprice correlation matrix
#     Only including characteristics with correlation values over corrMin (relative to SalePrice)

corrMin = 0.5
corrSalePrice = corrMat[corrMat['SalePrice'].abs() >= corrMin]['SalePrice'].sort_values(ascending = False)
print('Characteristics with correlation values over {}:\n{}\n'.format(corrMin, corrSalePrice.drop('SalePrice')))

cols = corrSalePrice.index
cm = np.corrcoef(df_train[cols].values.T)

sns.set(font_scale = 1.0, rc = {'figure.figsize': (10, 8)})
sns.heatmap(cm, cbar = True, annot = True, square = True,
            fmt = '0.2f', annot_kws = {'size': 10}, vmax = 0.8,
            xticklabels = cols.values, yticklabels = cols.values)

Referring to the previous strong correlations between characteristics (not inc. SalePrice):
* GarageArea (0.623423) & GarageCars (0.640473)
* TotRmsAbvGrd (0.533779) & GrLivArea (0.708618)
* 1stFlrSF (0.605968) & TotalBsmtSF (0.613905)

Therefore the following columns will be removed:
* GarageArea
* TotRmsAbvGrd
* 1stFlrSF

In [ ]:
df_train = df_train.drop(columns = ['GarageArea', 'TotRmsAbvGrd', '1stFlrSF'])
df_test = df_test.drop(columns = ['GarageArea', 'TotRmsAbvGrd', '1stFlrSF'])

And the following characteristics are worth exploring further:
* GrLivArea
* GarageCars
* TotalBsmtSF
* FullBath

## Initial data exploration

In [ ]:
# Check the size of the datasets
print('Size of the training dataset: {}'.format(df_train.shape))
print('Size of the testing dataset: {}\n'.format(df_test.shape))

# Make sure there are no zero-values in the SalePrice column
df_train['SalePrice'].describe()

There are no zero-values, which is great for modelling. However, the results above indicate that there will be a right skew.

In [ ]:
# Making these into functions for use later one

# Generate histogram, and print values for skewness and kurtosis on it
def histogramPlot(var, ax = None):
    p = 0
    if ax is None:
        ax = plt.gca()
        p = 1
    f = sns.distplot(df_train[var], fit = norm, ax = ax)
    f.ticklabel_format(style = 'plain', axis = 'y')
    ax.set(title = 'Distribution plot for {} values'.format(var),
            ylabel = 'Frequency')
    ax.add_artist(AnchoredText('Skewness: {:.4f}\nKurtosis: {:.4f}\n'.format(df_train[var].skew(), df_train[var].kurt()),
                                loc = 'upper right', frameon = False))
    
    # If not part of a subplot, print the figure
    if p == 1: plt.show()

    
# Generate probability plot (QQ-plot)
def probabilityPlot(var, ax = None):
    p = 0
    if ax is None:
        ax = plt.gca()
        p = 1
    f = probplot(df_train[var], plot = plt)
    ax.set(title = 'Probability plot for {} values'.format(var))
    
    # If not part of a subplot, print the figure
    if p == 1: plt.show()


# Print the histogram abd probability plots as a side-by-side subplot
def histAndProbPlot(var):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 6))
    plt.subplots_adjust(wspace = 0.4)
    histogramPlot(var, ax1)
    probabilityPlot(var, ax2)
    plt.show()

    
histAndProbPlot('SalePrice')

Right skew (positive) confirmed. Noted for later, so that I can apply log transformations to normalise SalePrice (and likely other numeric characteristics).

In [ ]:
# Making this into a function for use later one
def scatterPlot(xVar, yVar):
    plt.scatter(x = df_train[xVar], y = df_train[yVar])
    plt.title('Relationship between {} and {}'.format(xVar, yVar))
    plt.xlabel(xVar)
    plt.ylabel(yVar)
    plt.show()
    
# Numerical values
variables = ['GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath']
sns.set(font_scale = 1.0, rc = {'figure.figsize': (5, 5)})
for var in variables:
    scatterPlot(var, 'SalePrice')
    
# Categorical variables
#variables = ['OverallQual', 'YearBuilt', 'YearRemodAdd']
#for var in variables:
#    sns.boxplot(x = df_train[var], y = df_train['SalePrice'])
#    plt.title('Relationship between {} and {}'.format(var, 'SalePrice'))
#    plt.xlabel(var)
#    plt.ylabel('SalePrice')
#    plt.xticks(rotation = 90)
#    plt.show()

The relationships are as expected, though there are some outliers to remove. Whilst removing data can be hazardous, GrLivArea and TotalBsmtSF have obvious outliers to the right of the plot, and these will be excluded. However, I'll ignore outliers in characteristics not mentioned above (because their correlation to SalePrice is much lower, their outliers shouldn't have as much negative impact).

In [ ]:
# Two outliers within the GrLivArea characteristic
df_train = df_train.drop(df_train[(df_train['GrLivArea'] > 4500) & (df_train['SalePrice'] < 200000)].index)

# One outlier within the TotalBsmtSF characteristic
df_train = df_train.drop(df_train[(df_train['TotalBsmtSF'] > 6000) & (df_train['SalePrice'] < 200000)].index)

## Standardising skewed features

In [ ]:
# Numeric columns
types = df_train.dtypes[df_train.dtypes != 'object']
#types = types[~types.index.isin(['Id'])].index

# Skews in the training data
#varSkews = pd.Series([df_train[t].skew() for t in types], types).sort_values(ascending=False)

# Apply log transformation to highly skewed features (skew > 0.75)
#varToTransform = varSkews[abs(varSkews) > 0.75].index
#for var in varToTransform:
#    df_train[var] = np.log1p(df_train[var])
#    df_test[var] = np.log1p(df_test[var])